In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# unscaled, log-normalized counts, with conditions subsampled to the same number of cells 
# and 2000 highly variable genes calculated jointly across all perturbation conditions, including control, using scanpy28 with default parameters (Supplementary Methods)

In [3]:
import scanpy as sc
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import os
from anndata import read_h5ad
import sys
sys.path.append("..")
from src import *

will use the CPU to calculate the distance matrix.
will use the CPU to calculate the distance matrix.


In [4]:
data_path = "/data/bionets/datasets/scrnaseq_ji"

In [5]:
mcf = read_h5ad(os.path.join(data_path, 'mcfarland.hdf5'))

In [6]:
sc.pp.log1p(mcf)
sc.pp.highly_variable_genes(mcf, n_top_genes=2000)
mcf = mcf[:, mcf.var.highly_variable]

In [7]:
sc.pp.neighbors(mcf, n_pcs=0)

/data/bionets/je30bery/conda/envs/graphtool/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
g6 = list(np.unique([v for v in mcf.obs["perturbation"].values if v.endswith("_6")]))
g24 = list(np.unique([v for v in mcf.obs["perturbation"].values if v.endswith("_24")]))
test_groups = [g6, g24]
reference = "control"

In [ ]:
rosenbaum(mcf, group_by="perturbation", reference=reference, test_group=g6, use_nx=False, k=None)

/data/bionets/je30bery/rosenbaum_test/notebooks/../src/rosenbaum.py:150: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["XMatch_group"] = np.where(adata.obs[group_by].isin(test_group), "test", "reference")


XMatch_group
reference    541
test         392
Name: count, dtype: int64
using CPU to calculate distance matrix.
creating distance graph.
num samples 934 edges 436645
matching.
